This notebook is to scrap table from a wikipedia page and turn the data in a new dataframe.(Roy Abimanyu)

In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import pandas as pd
from urllib.request import urlopen


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

In [3]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]

In [4]:
df.drop(df.loc[df['Borough']== "Not assigned"].index, inplace=True)

In [5]:
df.groupby('Postal code').agg(lambda x: ','.join(x))
dp = df.reset_index(drop=True)

In [6]:
dp.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [7]:
geocode = pd.read_csv('https://cocl.us/Geospatial_data')
geocode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
postal_geocode = pd.merge(dp, geocode, how='left', left_on = 'Postal code', right_on = 'Postal Code')
postal_geocode.drop("Postal Code", axis=1, inplace=True)
postal_geocode.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [9]:
postal_geocode

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509


First, I used the previous task coding and data and pasted in this notebook. I continued to get the geo coordinates provided in the submission page. I parsed the geo coordinates into a dataframe and merged it to the dataframe from the first task. I dropped the Postal Code column of the geo coordinates dataframe. 

## Clustering

In [10]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize  
import folium 
from sklearn.cluster import KMeans

In [11]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

In [13]:
for lat, lng, borough, neighborhood in zip(
        postal_geocode['Latitude'], 
        postal_geocode['Longitude'], 
        postal_geocode['Borough'], 
        postal_geocode['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='brown',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [14]:
## use Toronto to narrowing the borough
downtowndf = postal_geocode[postal_geocode['Borough'].str.contains("Toronto")].reset_index(drop=True)
DownTown = pd.DataFrame(downtowndf)

In [15]:
DownTown.shape

(39, 5)

In [16]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        DownTown['Latitude'], 
        DownTown['Longitude'], 
        DownTown['Borough'], 
        DownTown['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.5,
        parse_html=False).add_to(map_downtown)  

map_downtown

In [17]:
# Divide into 5 clusters
k=5
DownTown_Cluster = DownTown.drop(['Postal code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(DownTown_Cluster)
kmeans.labels_
DownTown.insert(0, 'Cluster Labels', kmeans.labels_)

In [21]:
DownTownCluster_Map = folium.Map(location=[43.651070,-79.347015],zoom_start=12)
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(DownTown['Latitude'], DownTown['Longitude'], DownTown['Neighborhood'], DownTown['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=50,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.2).add_to(DownTownCluster_Map)
       
DownTownCluster_Map

I used previous task in geo coordinates. Using the word 'Toronto' to narrowdown the borough, after looking at the projected dataframe in the map.
Coloring needs to be better, so I need to learn more python libraries colorscheme. I am also wondering whether the coordinates can be used as the coverage of neighborhood, but only work from the designated scale/zoom. Zooming in, the the coverage maintain its size.